#### Criando banco de dados

In [ ]:
import sqlite3

conn = sqlite3.connect('stackoverflow.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS questions(
    id INTERGER PRIMARY KEY,
    title TEXT,
    tag TEXT,
    body TEXT 
)
''')

conn.commit()
print('Banco de dados criado com sucesso')

#### Função que adiciona os dados a base

In [25]:
def save_question(questions,tag):
    for question in questions:
        cursor.execute(''' INSERT OR REPLACE INTO questions (id,title,tag,body) VALUES (?,?,?,?) ''',
            (question['question_id'],question['title'],tag,question['body']))
    conn.commit()
    print(f'Tema {question} salvo na base')


In [26]:
import httpx


url = "https://api.stackexchange.com/2.3/questions"
site = "stackoverflow"
api_key = "rl_sVA4bTddeGTtj2EFpzt2RKu7r"
page_size = 100
max_page = 5
assuntos = ['pandas','tensorflow','machine-learning','dataframe','scikit-learn',
            'data-visualization']

def get_stackoverflow_questions(tag, page):
    params = {
        "order": "desc",
        "sort": "activity",
        "tagged": tag,
        "site": site,
        "page": page,
        "filter" : "withbody",
        "page_size" : page_size,
        "key" : api_key,
        "is_answered" : 'True'
    }
    response = httpx.get(url, params=params)
    response.raise_for_status()  # Levanta um erro se a requisição falhar
    return response.json()

# Exemplo de uso

base = {
    'pandas' : [],
    'tensorflow': [],
    'machine-learning': [],
    'dataframe' : [],
    'scikit-learn' : [],
    'data-visualization' : []
}

for assunto in assuntos:
  for i in range(1,max_page+1):
      question = get_stackoverflow_questions(assunto, i)
      for item in question['items']:
        body ={
          'question_id' : item['question_id'],
          'title' : item['title'],
          'body' :  item['body']
        }
        base[assunto].append(body)


for tema in base:
    save_question(base[tema],tema)

Tema {'question_id': 78990048, 'title': 'Make borders in dataframe pandas', 'body': '<p>How to make borders in dataframe in each cell:\n<img src="https://i.sstatic.net/XIPAjA0c.png" alt="enter image description here" /></p>\n<p>For example I have got the following dataframe:</p>\n<pre><code>import pandas as pd\n\ndf = pd.DataFrame({\'planet\': [\'Earth\', \'Moon\', \'Mars\'], \'mass_to_earth\': [1, 0.606, 0.107]})\n\ndf\n</code></pre>\n<p>And I need to get bold (and not) borders in this table in each cell. How can I do this?</p>\n'} salvo na base
Tema {'question_id': 78954080, 'title': 'How to interpret the look_ahead_mask when training a generative transformer in tensorflow (2.10.1)?', 'body': '<p>I\'m trying to implement a training loop for a transformer, using an encoder/decoder pair. Based on my understanding of how transformers work, I was expecting that to generate my output sequence, I would have to iterate the decoder, feeding its output back as input on each step, and building